Weak Pin Bar: The volumes on a pin bar are important but the most important thing which determines if the pin bar is weak or not is the follow up bar and it's volume.

Hypothesis: If the follow up bar gives greater move than the pin bar but the volume was lower than the pin bar then, the low of the pin bar should get tested. This testing depends on the time frame of the pin bar. The results can be seen more quickly on intraday time frames so, we will be using 3, 5, and 15 minutes time frames on BTCUSDT and ETHUSDT futures contracts.

In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [105]:
# Working on 15 minutes time frame
btcusdt_15min_data = pd.read_csv("BTCUSDT_15min.csv")
btcusdt_15min_df = pd.DataFrame(btcusdt_15min_data)
btcusdt_15min_df.head()

,timestamp,open,high,low,close,volume
0,2022-07-04 20:15:00,19851.3,19853.6,19719.5,19776.6,4522.198
1,2022-07-04 20:30:00,19776.7,19779.3,19735.5,19758.5,2417.909
2,2022-07-04 20:45:00,19758.5,19809.2,19745.7,19774.4,2972.344
3,2022-07-04 21:00:00,19774.3,19790.0,19740.4,19789.9,3271.586
4,2022-07-04 21:15:00,19789.9,19811.4,19768.3,19773.8,2151.420


Checking for bullish pin bars on 15 minutes bars of BTCUSDT Futures

In [106]:
# Defining function to identify bullish pin bars
def is_bullish_pin_bar(row):
    o_price = row['open']
    h_price = row['high']
    l_price = row['low']
    c_price = row['close']

    body = abs(c_price - o_price)
    upper_wick = h_price - max(c_price, o_price)
    lower_wick = min(o_price, c_price) - l_price
    price_range = h_price - l_price
    if(price_range == 0):
        return False
    body_ratio = body/price_range

    if(c_price >= o_price):
        if(upper_wick < lower_wick and body_ratio < 0.5):
            return True
        else:
            return False
    return False

btcusdt_15min_df['bullish_pin_bar'] = btcusdt_15min_df.apply(is_bullish_pin_bar, axis=1)
btcusdt_15min_df.head()

,timestamp,open,high,low,close,volume,bullish_pin_bar
0,2022-07-04 20:15:00,19851.3,19853.6,19719.5,19776.6,4522.198,False
1,2022-07-04 20:30:00,19776.7,19779.3,19735.5,19758.5,2417.909,False
2,2022-07-04 20:45:00,19758.5,19809.2,19745.7,19774.4,2972.344,False
3,2022-07-04 21:00:00,19774.3,19790.0,19740.4,19789.9,3271.586,True
4,2022-07-04 21:15:00,19789.9,19811.4,19768.3,19773.8,2151.420,False


In [107]:
# Adding the follow up status to pin bars
follow_through = []
for i in range(len(btcusdt_15min_df) - 1):
    if(btcusdt_15min_df.loc[i, 'bullish_pin_bar']):
        pin_bar_volume = btcusdt_15min_df.loc[i, 'volume']
        next_bar_volume = btcusdt_15min_df.loc[i+1, 'volume']
        pin_bar_move = btcusdt_15min_df.loc[i,'close'] - btcusdt_15min_df.loc[i,'open']
        next_bar_move = btcusdt_15min_df.loc[i+1,'close'] - btcusdt_15min_df.loc[i+1,'open']
        if(next_bar_move > pin_bar_move and next_bar_volume < pin_bar_volume):
            follow_through.append('True')
        else:
            follow_through.append('False')
    else:
        follow_through.append('False')
follow_through.append('False')
btcusdt_15min_df['Weak follow up'] = follow_through
btcusdt_15min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up
105114,2025-07-03 18:45:00,109558.0,109637.9,109488.8,109637.8,406.823,False,False
105115,2025-07-03 19:00:00,109637.8,109825.0,109581.5,109749.8,1436.947,False,False
105116,2025-07-03 19:15:00,109749.8,109865.9,109709.5,109824.9,681.060,False,False
105117,2025-07-03 19:30:00,109824.9,109825.0,109620.8,109771.3,582.974,False,False
105118,2025-07-03 19:45:00,109771.3,109771.3,109628.0,109743.9,462.942,False,False


In [108]:
# Checking if lows of weak pin bars got tested
lows_tested = ['False']*len(btcusdt_15min_df)
for i in range(len(btcusdt_15min_df) - 1):
    if(btcusdt_15min_df.loc[i, 'Weak follow up'] == 'True'):
        pin_bar_low = btcusdt_15min_df.loc[i, 'low']
        for j in range(i+1, len(btcusdt_15min_df)):
            if(btcusdt_15min_df.loc[j, 'low'] < pin_bar_low):
                lows_tested[i] = 'True'
                break
btcusdt_15min_df['Weak Low Tested'] = lows_tested
btcusdt_15min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested
105114,2025-07-03 18:45:00,109558.0,109637.9,109488.8,109637.8,406.823,False,False,False
105115,2025-07-03 19:00:00,109637.8,109825.0,109581.5,109749.8,1436.947,False,False,False
105116,2025-07-03 19:15:00,109749.8,109865.9,109709.5,109824.9,681.060,False,False,False
105117,2025-07-03 19:30:00,109824.9,109825.0,109620.8,109771.3,582.974,False,False,False
105118,2025-07-03 19:45:00,109771.3,109771.3,109628.0,109743.9,462.942,False,False,False


In [109]:
# Defining bearish pin bars
def is_bearish_pin_bar(row):
    o_price = row['open']
    h_price = row['high']
    l_price = row['low']
    c_price = row['close']

    body = abs(c_price - o_price)
    upper_wick = h_price - max(c_price, o_price)
    lower_wick = min(o_price, c_price) - l_price
    price_range = h_price - l_price
    if(price_range == 0):
        return False
    body_ratio = body/price_range

    if(c_price <= o_price):
        if(upper_wick > lower_wick and body_ratio < 0.5):
            return True
        else:
            return False
    return False

btcusdt_15min_df['bearish_pin_bar'] = btcusdt_15min_df.apply(is_bearish_pin_bar, axis=1)
btcusdt_15min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested,bearish_pin_bar
105114,2025-07-03 18:45:00,109558.0,109637.9,109488.8,109637.8,406.823,False,False,False,False
105115,2025-07-03 19:00:00,109637.8,109825.0,109581.5,109749.8,1436.947,False,False,False,False
105116,2025-07-03 19:15:00,109749.8,109865.9,109709.5,109824.9,681.060,False,False,False,False
105117,2025-07-03 19:30:00,109824.9,109825.0,109620.8,109771.3,582.974,False,False,False,False
105118,2025-07-03 19:45:00,109771.3,109771.3,109628.0,109743.9,462.942,False,False,False,False


In [110]:
# Checking for weak follow down
follow_through = []
for i in range(len(btcusdt_15min_df) - 1):
    if(btcusdt_15min_df.loc[i, 'bearish_pin_bar']):
        pin_bar_volume = btcusdt_15min_df.loc[i, 'volume']
        next_bar_volume = btcusdt_15min_df.loc[i+1, 'volume']
        pin_bar_move = btcusdt_15min_df.loc[i,'open'] - btcusdt_15min_df.loc[i,'close']
        next_bar_move = btcusdt_15min_df.loc[i+1,'open'] - btcusdt_15min_df.loc[i+1,'close']
        if(next_bar_move > pin_bar_move and next_bar_volume < pin_bar_volume):
            follow_through.append('True')
        else:
            follow_through.append('False')
    else:
        follow_through.append('False')
follow_through.append('False')
btcusdt_15min_df['Weak follow down'] = follow_through
btcusdt_15min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested,bearish_pin_bar,Weak follow down
105114,2025-07-03 18:45:00,109558.0,109637.9,109488.8,109637.8,406.823,False,False,False,False,False
105115,2025-07-03 19:00:00,109637.8,109825.0,109581.5,109749.8,1436.947,False,False,False,False,False
105116,2025-07-03 19:15:00,109749.8,109865.9,109709.5,109824.9,681.060,False,False,False,False,False
105117,2025-07-03 19:30:00,109824.9,109825.0,109620.8,109771.3,582.974,False,False,False,False,False
105118,2025-07-03 19:45:00,109771.3,109771.3,109628.0,109743.9,462.942,False,False,False,False,False


In [111]:
# Checking if highs of weak pin bars got tested
highs_tested = ['False']*len(btcusdt_15min_df)
for i in range(len(btcusdt_15min_df) - 1):
    if(btcusdt_15min_df.loc[i, 'Weak follow down'] == 'True'):
        pin_bar_high = btcusdt_15min_df.loc[i, 'high']
        for j in range(i+1, len(btcusdt_15min_df)):
            if(btcusdt_15min_df.loc[j, 'high'] > pin_bar_high):
                highs_tested[i] = 'True'
                break
btcusdt_15min_df['Weak High Tested'] = highs_tested
btcusdt_15min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested,bearish_pin_bar,Weak follow down,Weak High Tested
105114,2025-07-03 18:45:00,109558.0,109637.9,109488.8,109637.8,406.823,False,False,False,False,False,False
105115,2025-07-03 19:00:00,109637.8,109825.0,109581.5,109749.8,1436.947,False,False,False,False,False,False
105116,2025-07-03 19:15:00,109749.8,109865.9,109709.5,109824.9,681.060,False,False,False,False,False,False
105117,2025-07-03 19:30:00,109824.9,109825.0,109620.8,109771.3,582.974,False,False,False,False,False,False
105118,2025-07-03 19:45:00,109771.3,109771.3,109628.0,109743.9,462.942,False,False,False,False,False,False


In [112]:
# Saving the complete dataframe as a csv
btcusdt_15min_df.to_csv('b_15min_analyzed.csv')

In [113]:
# Working on 5 minutes time frame
btcusdt_5min_data = pd.read_csv('BTCUSDT_5min.csv')
btcusdt_5min_df = pd.DataFrame(btcusdt_5min_data)
btcusdt_5min_df.head()

,timestamp,open,high,low,close,volume
0,2022-07-04 20:15:00,19851.3,19853.6,19823.4,19827.0,483.921
1,2022-07-04 20:20:00,19827.1,19842.4,19719.5,19746.0,2721.178
2,2022-07-04 20:25:00,19746.0,19793.6,19736.1,19776.6,1317.099
3,2022-07-04 20:30:00,19776.7,19776.7,19735.5,19763.2,914.103
4,2022-07-04 20:35:00,19763.1,19770.1,19740.0,19768.0,666.151


In [114]:
#Getting the bullish pin bars
btcusdt_5min_df['bullish_pin_bar'] = btcusdt_5min_df.apply(is_bullish_pin_bar, axis=1)
btcusdt_5min_df.head()

,timestamp,open,high,low,close,volume,bullish_pin_bar
0,2022-07-04 20:15:00,19851.3,19853.6,19823.4,19827.0,483.921,False
1,2022-07-04 20:20:00,19827.1,19842.4,19719.5,19746.0,2721.178,False
2,2022-07-04 20:25:00,19746.0,19793.6,19736.1,19776.6,1317.099,False
3,2022-07-04 20:30:00,19776.7,19776.7,19735.5,19763.2,914.103,False
4,2022-07-04 20:35:00,19763.1,19770.1,19740.0,19768.0,666.151,True


In [115]:
# Adding the follow up status to pin bars
follow_through = []
for i in range(len(btcusdt_5min_df) - 1):
    if(btcusdt_5min_df.loc[i, 'bullish_pin_bar']):
        pin_bar_volume = btcusdt_5min_df.loc[i, 'volume']
        next_bar_volume = btcusdt_5min_df.loc[i+1, 'volume']
        pin_bar_move = btcusdt_5min_df.loc[i,'close'] - btcusdt_5min_df.loc[i,'open']
        next_bar_move = btcusdt_5min_df.loc[i+1,'close'] - btcusdt_5min_df.loc[i+1,'open']
        if(next_bar_move > pin_bar_move and next_bar_volume < pin_bar_volume):
            follow_through.append('True')
        else:
            follow_through.append('False')
    else:
        follow_through.append('False')
follow_through.append('False')
btcusdt_5min_df['Weak follow up'] = follow_through
btcusdt_5min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up
315355,2025-07-03 19:50:00,109687.3,109724.9,109628.0,109686.1,152.176,False,False
315356,2025-07-03 19:55:00,109686.1,109767.8,109686.1,109743.9,225.817,False,False
315357,2025-07-03 20:00:00,109743.8,109816.9,109743.8,109813.6,172.753,False,False
315358,2025-07-03 20:05:00,109813.7,109837.0,109784.7,109836.9,191.596,True,False
315359,2025-07-03 20:10:00,109836.9,109854.2,109750.2,109766.2,165.509,False,False


In [116]:
# Checking if lows of weak pin bars got tested
lows_tested = ['False']*len(btcusdt_5min_df)
for i in range(len(btcusdt_5min_df) - 1):
    if(btcusdt_5min_df.loc[i, 'Weak follow up'] == 'True'):
        pin_bar_low = btcusdt_5min_df.loc[i, 'low']
        for j in range(i+1, len(btcusdt_5min_df)):
            if(btcusdt_5min_df.loc[j, 'low'] < pin_bar_low):
                lows_tested[i] = 'True'
                break
btcusdt_5min_df['Weak Low Tested'] = lows_tested
btcusdt_5min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested
315355,2025-07-03 19:50:00,109687.3,109724.9,109628.0,109686.1,152.176,False,False,False
315356,2025-07-03 19:55:00,109686.1,109767.8,109686.1,109743.9,225.817,False,False,False
315357,2025-07-03 20:00:00,109743.8,109816.9,109743.8,109813.6,172.753,False,False,False
315358,2025-07-03 20:05:00,109813.7,109837.0,109784.7,109836.9,191.596,True,False,False
315359,2025-07-03 20:10:00,109836.9,109854.2,109750.2,109766.2,165.509,False,False,False


In [117]:
#Getting bearish pin bars
btcusdt_5min_df['bearish_pin_bar'] = btcusdt_5min_df.apply(is_bearish_pin_bar, axis=1)
btcusdt_5min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested,bearish_pin_bar
315355,2025-07-03 19:50:00,109687.3,109724.9,109628.0,109686.1,152.176,False,False,False,False
315356,2025-07-03 19:55:00,109686.1,109767.8,109686.1,109743.9,225.817,False,False,False,False
315357,2025-07-03 20:00:00,109743.8,109816.9,109743.8,109813.6,172.753,False,False,False,False
315358,2025-07-03 20:05:00,109813.7,109837.0,109784.7,109836.9,191.596,True,False,False,False
315359,2025-07-03 20:10:00,109836.9,109854.2,109750.2,109766.2,165.509,False,False,False,False


In [118]:
# Checking for weak follow down
follow_through = []
for i in range(len(btcusdt_5min_df) - 1):
    if(btcusdt_5min_df.loc[i, 'bearish_pin_bar']):
        pin_bar_volume = btcusdt_5min_df.loc[i, 'volume']
        next_bar_volume = btcusdt_5min_df.loc[i+1, 'volume']
        pin_bar_move = btcusdt_5min_df.loc[i,'open'] - btcusdt_5min_df.loc[i,'close']
        next_bar_move = btcusdt_5min_df.loc[i+1,'open'] - btcusdt_5min_df.loc[i+1,'close']
        if(next_bar_move > pin_bar_move and next_bar_volume < pin_bar_volume):
            follow_through.append('True')
        else:
            follow_through.append('False')
    else:
        follow_through.append('False')
follow_through.append('False')
btcusdt_5min_df['Weak follow down'] = follow_through
btcusdt_5min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested,bearish_pin_bar,Weak follow down
315355,2025-07-03 19:50:00,109687.3,109724.9,109628.0,109686.1,152.176,False,False,False,False,False
315356,2025-07-03 19:55:00,109686.1,109767.8,109686.1,109743.9,225.817,False,False,False,False,False
315357,2025-07-03 20:00:00,109743.8,109816.9,109743.8,109813.6,172.753,False,False,False,False,False
315358,2025-07-03 20:05:00,109813.7,109837.0,109784.7,109836.9,191.596,True,False,False,False,False
315359,2025-07-03 20:10:00,109836.9,109854.2,109750.2,109766.2,165.509,False,False,False,False,False


In [119]:
# Checking if highs of weak pin bars got tested
highs_tested = ['False']*len(btcusdt_5min_df)
for i in range(len(btcusdt_5min_df) - 1):
    if(btcusdt_5min_df.loc[i, 'Weak follow down'] == 'True'):
        pin_bar_high = btcusdt_5min_df.loc[i, 'high']
        for j in range(i+1, len(btcusdt_5min_df)):
            if(btcusdt_5min_df.loc[j, 'high'] > pin_bar_high):
                highs_tested[i] = 'True'
                break
btcusdt_5min_df['Weak High Tested'] = highs_tested
btcusdt_5min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested,bearish_pin_bar,Weak follow down,Weak High Tested
315355,2025-07-03 19:50:00,109687.3,109724.9,109628.0,109686.1,152.176,False,False,False,False,False,False
315356,2025-07-03 19:55:00,109686.1,109767.8,109686.1,109743.9,225.817,False,False,False,False,False,False
315357,2025-07-03 20:00:00,109743.8,109816.9,109743.8,109813.6,172.753,False,False,False,False,False,False
315358,2025-07-03 20:05:00,109813.7,109837.0,109784.7,109836.9,191.596,True,False,False,False,False,False
315359,2025-07-03 20:10:00,109836.9,109854.2,109750.2,109766.2,165.509,False,False,False,False,False,False


In [120]:
# Saving the complete dataframe as a csv
btcusdt_5min_df.to_csv('b_5min_analyzed.csv')

In [121]:
# Working on 3 minutes time frame
btcusdt_3min_data = pd.read_csv('BTCUSDT_3min.csv')
btcusdt_3min_df = pd.DataFrame(btcusdt_3min_data)
btcusdt_3min_df.head()

,timestamp,open,high,low,close,volume
0,2022-07-04 20:21:00,19817.9,19817.9,19782.8,19786.0,763.841
1,2022-07-04 20:24:00,19785.9,19785.9,19719.5,19775.9,2424.585
2,2022-07-04 20:27:00,19776.0,19793.6,19768.2,19776.6,647.572
3,2022-07-04 20:30:00,19776.7,19776.7,19735.5,19765.2,714.112
4,2022-07-04 20:33:00,19765.1,19769.6,19754.1,19758.3,249.910


In [122]:
#Getting the bullish pin bars
btcusdt_3min_df['bullish_pin_bar'] = btcusdt_3min_df.apply(is_bullish_pin_bar, axis=1)
btcusdt_3min_df.head()

,timestamp,open,high,low,close,volume,bullish_pin_bar
0,2022-07-04 20:21:00,19817.9,19817.9,19782.8,19786.0,763.841,False
1,2022-07-04 20:24:00,19785.9,19785.9,19719.5,19775.9,2424.585,False
2,2022-07-04 20:27:00,19776.0,19793.6,19768.2,19776.6,647.572,False
3,2022-07-04 20:30:00,19776.7,19776.7,19735.5,19765.2,714.112,False
4,2022-07-04 20:33:00,19765.1,19769.6,19754.1,19758.3,249.910,False


In [123]:
# Adding the follow up status to pin bars
follow_through = []
for i in range(len(btcusdt_3min_df) - 1):
    if(btcusdt_3min_df.loc[i, 'bullish_pin_bar']):
        pin_bar_volume = btcusdt_3min_df.loc[i, 'volume']
        next_bar_volume = btcusdt_3min_df.loc[i+1, 'volume']
        pin_bar_move = btcusdt_3min_df.loc[i,'close'] - btcusdt_3min_df.loc[i,'open']
        next_bar_move = btcusdt_3min_df.loc[i+1,'close'] - btcusdt_3min_df.loc[i+1,'open']
        if(next_bar_move > pin_bar_move and next_bar_volume < pin_bar_volume):
            follow_through.append('True')
        else:
            follow_through.append('False')
    else:
        follow_through.append('False')
follow_through.append('False')
btcusdt_3min_df['Weak follow up'] = follow_through
btcusdt_3min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up
525596,2025-07-03 20:09:00,109801.6,109837.2,109801.6,109816.2,97.989,False,False
525597,2025-07-03 20:12:00,109816.3,109854.2,109750.2,109766.2,119.961,False,False
525598,2025-07-03 20:15:00,109766.2,109781.9,109711.2,109781.9,84.919,True,False
525599,2025-07-03 20:18:00,109781.8,109804.9,109725.0,109725.0,77.384,False,False
525600,2025-07-03 20:21:00,109725.0,109797.1,109725.0,109747.1,88.842,False,False


In [124]:
# Checking if lows of weak pin bars got tested
lows_tested = ['False']*len(btcusdt_3min_df)
for i in range(len(btcusdt_3min_df) - 1):
    if(btcusdt_3min_df.loc[i, 'Weak follow up'] == 'True'):
        pin_bar_low = btcusdt_3min_df.loc[i, 'low']
        for j in range(i+1, len(btcusdt_3min_df)):
            if(btcusdt_3min_df.loc[j, 'low'] < pin_bar_low):
                lows_tested[i] = 'True'
                break
btcusdt_3min_df['Weak Low Tested'] = lows_tested
btcusdt_3min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested
525596,2025-07-03 20:09:00,109801.6,109837.2,109801.6,109816.2,97.989,False,False,False
525597,2025-07-03 20:12:00,109816.3,109854.2,109750.2,109766.2,119.961,False,False,False
525598,2025-07-03 20:15:00,109766.2,109781.9,109711.2,109781.9,84.919,True,False,False
525599,2025-07-03 20:18:00,109781.8,109804.9,109725.0,109725.0,77.384,False,False,False
525600,2025-07-03 20:21:00,109725.0,109797.1,109725.0,109747.1,88.842,False,False,False


In [125]:
#Getting bearish pin bars
btcusdt_3min_df['bearish_pin_bar'] = btcusdt_3min_df.apply(is_bearish_pin_bar, axis=1)
btcusdt_3min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested,bearish_pin_bar
525596,2025-07-03 20:09:00,109801.6,109837.2,109801.6,109816.2,97.989,False,False,False,False
525597,2025-07-03 20:12:00,109816.3,109854.2,109750.2,109766.2,119.961,False,False,False,True
525598,2025-07-03 20:15:00,109766.2,109781.9,109711.2,109781.9,84.919,True,False,False,False
525599,2025-07-03 20:18:00,109781.8,109804.9,109725.0,109725.0,77.384,False,False,False,False
525600,2025-07-03 20:21:00,109725.0,109797.1,109725.0,109747.1,88.842,False,False,False,False


In [126]:
# Checking for weak follow down
follow_through = []
for i in range(len(btcusdt_3min_df) - 1):
    if(btcusdt_3min_df.loc[i, 'bearish_pin_bar']):
        pin_bar_volume = btcusdt_3min_df.loc[i, 'volume']
        next_bar_volume = btcusdt_3min_df.loc[i+1, 'volume']
        pin_bar_move = btcusdt_3min_df.loc[i,'open'] - btcusdt_3min_df.loc[i,'close']
        next_bar_move = btcusdt_3min_df.loc[i+1,'open'] - btcusdt_3min_df.loc[i+1,'close']
        if(next_bar_move > pin_bar_move and next_bar_volume < pin_bar_volume):
            follow_through.append('True')
        else:
            follow_through.append('False')
    else:
        follow_through.append('False')
follow_through.append('False')
btcusdt_3min_df['Weak follow down'] = follow_through
btcusdt_3min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested,bearish_pin_bar,Weak follow down
525596,2025-07-03 20:09:00,109801.6,109837.2,109801.6,109816.2,97.989,False,False,False,False,False
525597,2025-07-03 20:12:00,109816.3,109854.2,109750.2,109766.2,119.961,False,False,False,True,False
525598,2025-07-03 20:15:00,109766.2,109781.9,109711.2,109781.9,84.919,True,False,False,False,False
525599,2025-07-03 20:18:00,109781.8,109804.9,109725.0,109725.0,77.384,False,False,False,False,False
525600,2025-07-03 20:21:00,109725.0,109797.1,109725.0,109747.1,88.842,False,False,False,False,False


In [127]:
# Checking if highs of weak pin bars got tested
highs_tested = ['False']*len(btcusdt_3min_df)
for i in range(len(btcusdt_3min_df) - 1):
    if(btcusdt_3min_df.loc[i, 'Weak follow down'] == 'True'):
        pin_bar_high = btcusdt_3min_df.loc[i, 'high']
        for j in range(i+1, len(btcusdt_3min_df)):
            if(btcusdt_3min_df.loc[j, 'high'] > pin_bar_high):
                highs_tested[i] = 'True'
                break
btcusdt_3min_df['Weak High Tested'] = highs_tested
btcusdt_3min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested,bearish_pin_bar,Weak follow down,Weak High Tested
525596,2025-07-03 20:09:00,109801.6,109837.2,109801.6,109816.2,97.989,False,False,False,False,False,False
525597,2025-07-03 20:12:00,109816.3,109854.2,109750.2,109766.2,119.961,False,False,False,True,False,False
525598,2025-07-03 20:15:00,109766.2,109781.9,109711.2,109781.9,84.919,True,False,False,False,False,False
525599,2025-07-03 20:18:00,109781.8,109804.9,109725.0,109725.0,77.384,False,False,False,False,False,False
525600,2025-07-03 20:21:00,109725.0,109797.1,109725.0,109747.1,88.842,False,False,False,False,False,False


In [128]:
# Saving the complete dataframe as a csv
btcusdt_3min_df.to_csv('b_3min_analyzed.csv')

In [129]:
# Starting with ethusdt 15 minutes
ethusdt_15min_data = pd.read_csv('ETHUSDT_15min.csv')
ethusdt_15min_df = pd.DataFrame(ethusdt_15min_data)
ethusdt_15min_df.head()

,timestamp,open,high,low,close,volume
0,2022-07-04 20:45:00,1119.19,1124.05,1118.50,1121.29,27557.359
1,2022-07-04 21:00:00,1121.29,1123.41,1119.14,1123.36,17761.827
2,2022-07-04 21:15:00,1123.35,1125.17,1120.46,1120.86,17180.641
3,2022-07-04 21:30:00,1120.86,1134.23,1120.30,1126.59,49539.554
4,2022-07-04 21:45:00,1126.59,1150.00,1126.13,1134.54,99346.667


In [130]:
# Getting bullish bars 
ethusdt_15min_df['bullish_pin_bar'] = ethusdt_15min_df.apply(is_bullish_pin_bar, axis=1)
ethusdt_15min_df.head()

,timestamp,open,high,low,close,volume,bullish_pin_bar
0,2022-07-04 20:45:00,1119.19,1124.05,1118.50,1121.29,27557.359,False
1,2022-07-04 21:00:00,1121.29,1123.41,1119.14,1123.36,17761.827,True
2,2022-07-04 21:15:00,1123.35,1125.17,1120.46,1120.86,17180.641,False
3,2022-07-04 21:30:00,1120.86,1134.23,1120.30,1126.59,49539.554,False
4,2022-07-04 21:45:00,1126.59,1150.00,1126.13,1134.54,99346.667,False


In [131]:
# Adding the follow up status to pin bars
follow_through = []
for i in range(len(ethusdt_15min_df) - 1):
    if(ethusdt_15min_df.loc[i, 'bullish_pin_bar']):
        pin_bar_volume = ethusdt_15min_df.loc[i, 'volume']
        next_bar_volume = ethusdt_15min_df.loc[i+1, 'volume']
        pin_bar_move = ethusdt_15min_df.loc[i,'close'] - ethusdt_15min_df.loc[i,'open']
        next_bar_move = ethusdt_15min_df.loc[i+1,'close'] - ethusdt_15min_df.loc[i+1,'open']
        if(next_bar_move > pin_bar_move and next_bar_volume < pin_bar_volume):
            follow_through.append('True')
        else:
            follow_through.append('False')
    else:
        follow_through.append('False')
follow_through.append('False')
ethusdt_15min_df['Weak follow up'] = follow_through
ethusdt_15min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up
105114,2025-07-03 19:15:00,2588.60,2590.85,2587.03,2587.13,19852.263,False,False
105115,2025-07-03 19:30:00,2587.13,2587.43,2580.53,2584.60,18774.145,False,False
105116,2025-07-03 19:45:00,2584.59,2586.40,2582.44,2584.63,12267.453,True,False
105117,2025-07-03 20:00:00,2584.63,2592.63,2584.10,2588.91,21872.346,False,False
105118,2025-07-03 20:15:00,2588.91,2595.54,2586.82,2593.61,23658.499,False,False


In [132]:
# Checking if lows of weak pin bars got tested
lows_tested = ['False']*len(ethusdt_15min_df)
for i in range(len(ethusdt_15min_df) - 1):
    if(ethusdt_15min_df.loc[i, 'Weak follow up'] == 'True'):
        pin_bar_low = ethusdt_15min_df.loc[i, 'low']
        for j in range(i+1, len(ethusdt_15min_df)):
            if(ethusdt_15min_df.loc[j, 'low'] < pin_bar_low):
                lows_tested[i] = 'True'
                break
ethusdt_15min_df['Weak Low Tested'] = lows_tested
ethusdt_15min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested
105114,2025-07-03 19:15:00,2588.60,2590.85,2587.03,2587.13,19852.263,False,False,False
105115,2025-07-03 19:30:00,2587.13,2587.43,2580.53,2584.60,18774.145,False,False,False
105116,2025-07-03 19:45:00,2584.59,2586.40,2582.44,2584.63,12267.453,True,False,False
105117,2025-07-03 20:00:00,2584.63,2592.63,2584.10,2588.91,21872.346,False,False,False
105118,2025-07-03 20:15:00,2588.91,2595.54,2586.82,2593.61,23658.499,False,False,False


In [133]:
#Getting weak bearish pin bars
ethusdt_15min_df['bearish_pin_bar'] = ethusdt_15min_df.apply(is_bearish_pin_bar, axis=1)
ethusdt_15min_df.head()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested,bearish_pin_bar
0,2022-07-04 20:45:00,1119.19,1124.05,1118.50,1121.29,27557.359,False,False,False,False
1,2022-07-04 21:00:00,1121.29,1123.41,1119.14,1123.36,17761.827,True,False,False,False
2,2022-07-04 21:15:00,1123.35,1125.17,1120.46,1120.86,17180.641,False,False,False,False
3,2022-07-04 21:30:00,1120.86,1134.23,1120.30,1126.59,49539.554,False,False,False,False
4,2022-07-04 21:45:00,1126.59,1150.00,1126.13,1134.54,99346.667,False,False,False,False


In [134]:
# Checking for weak follow down
follow_through = []
for i in range(len(ethusdt_15min_df) - 1):
    if(ethusdt_15min_df.loc[i, 'bearish_pin_bar']):
        pin_bar_volume = ethusdt_15min_df.loc[i, 'volume']
        next_bar_volume = ethusdt_15min_df.loc[i+1, 'volume']
        pin_bar_move = ethusdt_15min_df.loc[i,'open'] - ethusdt_15min_df.loc[i,'close']
        next_bar_move = ethusdt_15min_df.loc[i+1,'open'] - ethusdt_15min_df.loc[i+1,'close']
        if(next_bar_move > pin_bar_move and next_bar_volume < pin_bar_volume):
            follow_through.append('True')
        else:
            follow_through.append('False')
    else:
        follow_through.append('False')
follow_through.append('False')
ethusdt_15min_df['Weak follow down'] = follow_through
ethusdt_15min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested,bearish_pin_bar,Weak follow down
105114,2025-07-03 19:15:00,2588.60,2590.85,2587.03,2587.13,19852.263,False,False,False,True,True
105115,2025-07-03 19:30:00,2587.13,2587.43,2580.53,2584.60,18774.145,False,False,False,False,False
105116,2025-07-03 19:45:00,2584.59,2586.40,2582.44,2584.63,12267.453,True,False,False,False,False
105117,2025-07-03 20:00:00,2584.63,2592.63,2584.10,2588.91,21872.346,False,False,False,False,False
105118,2025-07-03 20:15:00,2588.91,2595.54,2586.82,2593.61,23658.499,False,False,False,False,False


In [135]:
# Checking if highs of weak pin bars got tested
highs_tested = ['False']*len(ethusdt_15min_df)
for i in range(len(ethusdt_15min_df) - 1):
    if(ethusdt_15min_df.loc[i, 'Weak follow down'] == 'True'):
        pin_bar_high = ethusdt_15min_df.loc[i, 'high']
        for j in range(i+1, len(ethusdt_15min_df)):
            if(ethusdt_15min_df.loc[j, 'high'] > pin_bar_high):
                highs_tested[i] = 'True'
                break
ethusdt_15min_df['Weak High Tested'] = highs_tested
ethusdt_15min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested,bearish_pin_bar,Weak follow down,Weak High Tested
105114,2025-07-03 19:15:00,2588.60,2590.85,2587.03,2587.13,19852.263,False,False,False,True,True,True
105115,2025-07-03 19:30:00,2587.13,2587.43,2580.53,2584.60,18774.145,False,False,False,False,False,False
105116,2025-07-03 19:45:00,2584.59,2586.40,2582.44,2584.63,12267.453,True,False,False,False,False,False
105117,2025-07-03 20:00:00,2584.63,2592.63,2584.10,2588.91,21872.346,False,False,False,False,False,False
105118,2025-07-03 20:15:00,2588.91,2595.54,2586.82,2593.61,23658.499,False,False,False,False,False,False


In [136]:
#Saving the data frame into csv
ethusdt_15min_df.to_csv('e_15min_analyzed.csv')

In [137]:
#Starting the analysis for 5 minutes in ETHUSDT
ethusdt_5min_data = pd.read_csv('ETHUSDT_5min.csv')
ethusdt_5min_df = pd.DataFrame(ethusdt_5min_data)
ethusdt_5min_df.head()

,timestamp,open,high,low,close,volume
0,2022-07-04 20:35:00,1118.56,1119.46,1117.58,1118.89,5259.760
1,2022-07-04 20:40:00,1118.89,1119.43,1117.32,1119.20,7271.598
2,2022-07-04 20:45:00,1119.19,1122.60,1118.50,1121.56,14192.020
3,2022-07-04 20:50:00,1121.56,1124.05,1121.54,1122.78,6814.047
4,2022-07-04 20:55:00,1122.77,1123.00,1120.32,1121.29,6551.292


In [138]:
#Getting the weak bullish pin bars
ethusdt_5min_df['bullish_pin_bar'] = ethusdt_5min_df.apply(is_bullish_pin_bar, axis=1)
ethusdt_5min_df.head()

,timestamp,open,high,low,close,volume,bullish_pin_bar
0,2022-07-04 20:35:00,1118.56,1119.46,1117.58,1118.89,5259.760,True
1,2022-07-04 20:40:00,1118.89,1119.43,1117.32,1119.20,7271.598,True
2,2022-07-04 20:45:00,1119.19,1122.60,1118.50,1121.56,14192.020,False
3,2022-07-04 20:50:00,1121.56,1124.05,1121.54,1122.78,6814.047,False
4,2022-07-04 20:55:00,1122.77,1123.00,1120.32,1121.29,6551.292,False


In [139]:
#Finding the weak follow ups
follow_through = []
for i in range(len(ethusdt_5min_df) - 1):
    if(ethusdt_5min_df.loc[i, 'bullish_pin_bar']):
        pin_bar_volume = ethusdt_5min_df.loc[i, 'volume']
        next_bar_volume = ethusdt_5min_df.loc[i+1, 'volume']
        pin_bar_move = ethusdt_5min_df.loc[i,'close'] - ethusdt_5min_df.loc[i,'open']
        next_bar_move = ethusdt_5min_df.loc[i+1,'close'] - ethusdt_5min_df.loc[i+1,'open']
        if(next_bar_move > pin_bar_move and next_bar_volume < pin_bar_volume):
            follow_through.append('True')
        else:
            follow_through.append('False')
    else:
        follow_through.append('False')
follow_through.append('False')
ethusdt_5min_df['Weak follow up'] = follow_through
ethusdt_5min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up
315355,2025-07-03 20:10:00,2590.00,2592.63,2587.50,2588.91,11477.755,False,False
315356,2025-07-03 20:15:00,2588.91,2590.91,2586.82,2590.78,5668.784,True,False
315357,2025-07-03 20:20:00,2590.77,2592.23,2589.41,2591.36,6041.607,True,False
315358,2025-07-03 20:25:00,2591.36,2595.54,2591.35,2593.61,11948.108,False,False
315359,2025-07-03 20:30:00,2593.62,2593.88,2591.42,2591.68,4993.554,False,False


In [140]:
# Checking if lows of weak pin bars got tested
lows_tested = ['False']*len(ethusdt_5min_df)
for i in range(len(ethusdt_5min_df) - 1):
    if(ethusdt_5min_df.loc[i, 'Weak follow up'] == 'True'):
        pin_bar_low = ethusdt_5min_df.loc[i, 'low']
        for j in range(i+1, len(ethusdt_5min_df)):
            if(ethusdt_5min_df.loc[j, 'low'] < pin_bar_low):
                lows_tested[i] = 'True'
                break
ethusdt_5min_df['Weak Low Tested'] = lows_tested
ethusdt_5min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested
315355,2025-07-03 20:10:00,2590.00,2592.63,2587.50,2588.91,11477.755,False,False,False
315356,2025-07-03 20:15:00,2588.91,2590.91,2586.82,2590.78,5668.784,True,False,False
315357,2025-07-03 20:20:00,2590.77,2592.23,2589.41,2591.36,6041.607,True,False,False
315358,2025-07-03 20:25:00,2591.36,2595.54,2591.35,2593.61,11948.108,False,False,False
315359,2025-07-03 20:30:00,2593.62,2593.88,2591.42,2591.68,4993.554,False,False,False


In [141]:
#Getting bearish pin bars
ethusdt_5min_df['bearish_pin_bar'] = ethusdt_5min_df.apply(is_bearish_pin_bar, axis=1)
ethusdt_5min_df.head()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested,bearish_pin_bar
0,2022-07-04 20:35:00,1118.56,1119.46,1117.58,1118.89,5259.760,True,False,False,False
1,2022-07-04 20:40:00,1118.89,1119.43,1117.32,1119.20,7271.598,True,False,False,False
2,2022-07-04 20:45:00,1119.19,1122.60,1118.50,1121.56,14192.020,False,False,False,False
3,2022-07-04 20:50:00,1121.56,1124.05,1121.54,1122.78,6814.047,False,False,False,False
4,2022-07-04 20:55:00,1122.77,1123.00,1120.32,1121.29,6551.292,False,False,False,False


In [142]:
#Checking for weak follow down
follow_through = []
for i in range(len(ethusdt_5min_df) - 1):
    if(ethusdt_5min_df.loc[i, 'bearish_pin_bar']):
        pin_bar_volume = ethusdt_5min_df.loc[i, 'volume']
        next_bar_volume = ethusdt_5min_df.loc[i+1, 'volume']
        pin_bar_move = ethusdt_5min_df.loc[i,'open'] - ethusdt_5min_df.loc[i,'close']
        next_bar_move = ethusdt_5min_df.loc[i+1,'open'] - ethusdt_5min_df.loc[i+1,'close']
        if(next_bar_move > pin_bar_move and next_bar_volume < pin_bar_volume):
            follow_through.append('True')
        else:
            follow_through.append('False')
    else:
        follow_through.append('False')
follow_through.append('False')
ethusdt_5min_df['Weak follow down'] = follow_through
ethusdt_5min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested,bearish_pin_bar,Weak follow down
315355,2025-07-03 20:10:00,2590.00,2592.63,2587.50,2588.91,11477.755,False,False,False,True,False
315356,2025-07-03 20:15:00,2588.91,2590.91,2586.82,2590.78,5668.784,True,False,False,False,False
315357,2025-07-03 20:20:00,2590.77,2592.23,2589.41,2591.36,6041.607,True,False,False,False,False
315358,2025-07-03 20:25:00,2591.36,2595.54,2591.35,2593.61,11948.108,False,False,False,False,False
315359,2025-07-03 20:30:00,2593.62,2593.88,2591.42,2591.68,4993.554,False,False,False,False,False


In [143]:
# Checking if highs of weak pin bars got tested
highs_tested = ['False']*len(ethusdt_5min_df)
for i in range(len(ethusdt_5min_df) - 1):
    if(ethusdt_5min_df.loc[i, 'Weak follow down'] == 'True'):
        pin_bar_high = ethusdt_5min_df.loc[i, 'high']
        for j in range(i+1, len(ethusdt_5min_df)):
            if(ethusdt_5min_df.loc[j, 'high'] > pin_bar_high):
                highs_tested[i] = 'True'
                break
ethusdt_5min_df['Weak High Tested'] = highs_tested
ethusdt_5min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested,bearish_pin_bar,Weak follow down,Weak High Tested
315355,2025-07-03 20:10:00,2590.00,2592.63,2587.50,2588.91,11477.755,False,False,False,True,False,False
315356,2025-07-03 20:15:00,2588.91,2590.91,2586.82,2590.78,5668.784,True,False,False,False,False,False
315357,2025-07-03 20:20:00,2590.77,2592.23,2589.41,2591.36,6041.607,True,False,False,False,False,False
315358,2025-07-03 20:25:00,2591.36,2595.54,2591.35,2593.61,11948.108,False,False,False,False,False,False
315359,2025-07-03 20:30:00,2593.62,2593.88,2591.42,2591.68,4993.554,False,False,False,False,False,False


In [144]:
#Saving the data frame into csv
ethusdt_5min_df.to_csv('e_5min_analyzed.csv')

In [145]:
#Starting with 3 minutes time frame in ETHUSDT
ethusdt_3min_data = pd.read_csv('ETHUSDT_3min.csv')
ethusdt_3min_df = pd.DataFrame(ethusdt_3min_data)
ethusdt_3min_df.head()

,timestamp,open,high,low,close,volume
0,2022-07-04 20:30:00,1119.96,1120.08,1117.43,1118.92,5457.249
1,2022-07-04 20:33:00,1118.92,1119.27,1118.26,1118.44,2651.778
2,2022-07-04 20:36:00,1118.44,1118.73,1117.58,1117.95,2811.788
3,2022-07-04 20:39:00,1117.95,1119.46,1117.69,1118.23,3746.213
4,2022-07-04 20:42:00,1118.23,1119.43,1117.32,1119.20,5090.926


In [146]:
#Getting the bullish pin bars
ethusdt_3min_df['bullish_pin_bar'] = ethusdt_3min_df.apply(is_bullish_pin_bar, axis=1)
ethusdt_3min_df.head()

,timestamp,open,high,low,close,volume,bullish_pin_bar
0,2022-07-04 20:30:00,1119.96,1120.08,1117.43,1118.92,5457.249,False
1,2022-07-04 20:33:00,1118.92,1119.27,1118.26,1118.44,2651.778,False
2,2022-07-04 20:36:00,1118.44,1118.73,1117.58,1117.95,2811.788,False
3,2022-07-04 20:39:00,1117.95,1119.46,1117.69,1118.23,3746.213,False
4,2022-07-04 20:42:00,1118.23,1119.43,1117.32,1119.20,5090.926,True


In [147]:
#Finding weak follow ups
follow_through = []
for i in range(len(ethusdt_3min_df) - 1):
    if(ethusdt_3min_df.loc[i, 'bullish_pin_bar']):
        pin_bar_volume = ethusdt_3min_df.loc[i, 'volume']
        next_bar_volume = ethusdt_3min_df.loc[i+1, 'volume']
        pin_bar_move = ethusdt_3min_df.loc[i,'close'] - ethusdt_3min_df.loc[i,'open']
        next_bar_move = ethusdt_3min_df.loc[i+1,'close'] - ethusdt_3min_df.loc[i+1,'open']
        if(next_bar_move > pin_bar_move and next_bar_volume < pin_bar_volume):
            follow_through.append('True')
        else:
            follow_through.append('False')
    else:
        follow_through.append('False')
follow_through.append('False')
ethusdt_3min_df['Weak follow up'] = follow_through
ethusdt_3min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up
525596,2025-07-03 20:18:00,2589.80,2592.23,2589.10,2589.63,3256.317,False,False
525597,2025-07-03 20:21:00,2589.64,2591.95,2589.50,2589.78,3865.435,False,False
525598,2025-07-03 20:24:00,2589.77,2593.77,2589.75,2592.43,5697.873,False,False
525599,2025-07-03 20:27:00,2592.43,2595.54,2592.30,2593.61,7126.433,False,False
525600,2025-07-03 20:30:00,2593.62,2593.88,2592.21,2592.22,2624.863,False,False


In [148]:
# Checking if lows of weak pin bars got tested
lows_tested = ['False']*len(ethusdt_3min_df)
for i in range(len(ethusdt_3min_df) - 1):
    if(ethusdt_3min_df.loc[i, 'Weak follow up'] == 'True'):
        pin_bar_low = ethusdt_3min_df.loc[i, 'low']
        for j in range(i+1, len(ethusdt_3min_df)):
            if(ethusdt_3min_df.loc[j, 'low'] < pin_bar_low):
                lows_tested[i] = 'True'
                break
ethusdt_3min_df['Weak Low Tested'] = lows_tested
ethusdt_3min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested
525596,2025-07-03 20:18:00,2589.80,2592.23,2589.10,2589.63,3256.317,False,False,False
525597,2025-07-03 20:21:00,2589.64,2591.95,2589.50,2589.78,3865.435,False,False,False
525598,2025-07-03 20:24:00,2589.77,2593.77,2589.75,2592.43,5697.873,False,False,False
525599,2025-07-03 20:27:00,2592.43,2595.54,2592.30,2593.61,7126.433,False,False,False
525600,2025-07-03 20:30:00,2593.62,2593.88,2592.21,2592.22,2624.863,False,False,False


In [149]:
#Getting bearish pin bars
ethusdt_3min_df['bearish_pin_bar'] = ethusdt_3min_df.apply(is_bearish_pin_bar, axis=1)
ethusdt_3min_df.head()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested,bearish_pin_bar
0,2022-07-04 20:30:00,1119.96,1120.08,1117.43,1118.92,5457.249,False,False,False,False
1,2022-07-04 20:33:00,1118.92,1119.27,1118.26,1118.44,2651.778,False,False,False,True
2,2022-07-04 20:36:00,1118.44,1118.73,1117.58,1117.95,2811.788,False,False,False,False
3,2022-07-04 20:39:00,1117.95,1119.46,1117.69,1118.23,3746.213,False,False,False,False
4,2022-07-04 20:42:00,1118.23,1119.43,1117.32,1119.20,5090.926,True,False,False,False


In [150]:
#Finding weak follow downs
follow_through = []
for i in range(len(ethusdt_3min_df) - 1):
    if(ethusdt_3min_df.loc[i, 'bearish_pin_bar']):
        pin_bar_volume = ethusdt_3min_df.loc[i, 'volume']
        next_bar_volume = ethusdt_3min_df.loc[i+1, 'volume']
        pin_bar_move = ethusdt_3min_df.loc[i,'open'] - ethusdt_3min_df.loc[i,'close']
        next_bar_move = ethusdt_3min_df.loc[i+1,'open'] - ethusdt_3min_df.loc[i+1,'close']
        if(next_bar_move > pin_bar_move and next_bar_volume < pin_bar_volume):
            follow_through.append('True')
        else:
            follow_through.append('False')
    else:
        follow_through.append('False')
follow_through.append('False')
ethusdt_3min_df['Weak follow down'] = follow_through
ethusdt_3min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested,bearish_pin_bar,Weak follow down
525596,2025-07-03 20:18:00,2589.80,2592.23,2589.10,2589.63,3256.317,False,False,False,True,False
525597,2025-07-03 20:21:00,2589.64,2591.95,2589.50,2589.78,3865.435,False,False,False,False,False
525598,2025-07-03 20:24:00,2589.77,2593.77,2589.75,2592.43,5697.873,False,False,False,False,False
525599,2025-07-03 20:27:00,2592.43,2595.54,2592.30,2593.61,7126.433,False,False,False,False,False
525600,2025-07-03 20:30:00,2593.62,2593.88,2592.21,2592.22,2624.863,False,False,False,False,False


In [151]:
# Checking if highs of weak pin bars got tested
highs_tested = ['False']*len(ethusdt_3min_df)
for i in range(len(ethusdt_3min_df) - 1):
    if(ethusdt_3min_df.loc[i, 'Weak follow down'] == 'True'):
        pin_bar_high = ethusdt_3min_df.loc[i, 'high']
        for j in range(i+1, len(ethusdt_3min_df)):
            if(ethusdt_3min_df.loc[j, 'high'] > pin_bar_high):
                highs_tested[i] = 'True'
                break
ethusdt_3min_df['Weak High Tested'] = highs_tested
ethusdt_3min_df.tail()

,timestamp,open,high,low,close,volume,bullish_pin_bar,Weak follow up,Weak Low Tested,bearish_pin_bar,Weak follow down,Weak High Tested
525596,2025-07-03 20:18:00,2589.80,2592.23,2589.10,2589.63,3256.317,False,False,False,True,False,False
525597,2025-07-03 20:21:00,2589.64,2591.95,2589.50,2589.78,3865.435,False,False,False,False,False,False
525598,2025-07-03 20:24:00,2589.77,2593.77,2589.75,2592.43,5697.873,False,False,False,False,False,False
525599,2025-07-03 20:27:00,2592.43,2595.54,2592.30,2593.61,7126.433,False,False,False,False,False,False
525600,2025-07-03 20:30:00,2593.62,2593.88,2592.21,2592.22,2624.863,False,False,False,False,False,False


In [152]:
#Saving Data frame to csv
ethusdt_3min_df.to_csv('e_3min_analyzed.csv')